In [1]:
import csv

import rosbag
from cv_bridge import CvBridge

from pynsia.pointcloud import pc2_to_pc
from pynsia.ros.bag import topics

from bag_parsers import *
from settings import Paths

In [2]:
DATA_DIR = '/media/blazaid/Saca/Phd/data'

SUBJECTS_AND_DATASETS = [
    ('miguel', ('train', 'test')),
]

TOPICS_TO_EXTRACT = [
    '/capture/can',
    '/capture/gps/position',
    '/capture/gps/speed',
    '/capture/kinect/image',
    '/capture/lidar'
]

CAN_MESSAGES = [
    '696', '208', '412', '346', '374',
    '236', '2F2', '418', '424', '231'
]
KINECT_IMAGES_DIR = 'images'
LIDAR_POINTCLOUDS_DIR = 'lidar'

In [3]:
paths = Paths(DATA_DIR)
print('Extracting data from subjects ...')
for subject, datasets in SUBJECTS_AND_DATASETS:
    print('Subject:\t' + subject)
    for dataset in datasets:
        topics_parsers = {
            '/capture/can': CanParser(allowed_messages=CAN_MESSAGES),
            '/capture/gps/position': GpsPositionParser(),
            '/capture/gps/speed': GpsSpeedParser(),
            '/capture/kinect/image': KinectImageParser(paths.raw_csv_dir(
                subject,
                dataset,
                KINECT_IMAGES_DIR,
            )),
            '/capture/lidar': LidarParser(paths.raw_csv_dir(
                subject,
                dataset,
                LIDAR_POINTCLOUDS_DIR,
            )),
        }
        bag_dataset = paths.bag_dataset_path(subject, dataset)
        print('Loading bag:\t' + bag_dataset)
        bag = rosbag.Bag(bag_dataset)
        for topic in topics(bag):
            # Only proicess the extractable topics
            if topic not in TOPICS_TO_EXTRACT:
                print('Skipping topic:\t{}'.format(topic))
                continue
            else:
                print('Dumping ' + topic)
             
            # Get the parser for the topic
            parser = topics_parsers[topic]
            # Get the csv file to dump
            csv_file = paths.raw_csv_dataset_path(subject, dataset, topic)
            with open(csv_file, 'w+') as f:
                # Create the object that writes csv
                writer = csv.writer(f)
                # Start writing the topics as rows in the csv
                print('Writing raw csv:\t' + csv_file + ' ...')
                for i, (_, msg, ts) in enumerate(bag.read_messages(topic)):
                    # For each topic name, write headers
                    if i == 0:
                        writer.writerow(parser.header(msg))
                    # Write the row if there is any
                    row = parser.row(msg)
                    if row is not None:
                        writer.writerow(row)

Extracting data from subjects ...
Subject:	miguel
Loading bag:	/media/blazaid/Saca/Phd/data/bags/miguel-train.bag
Dumping /capture/can
Writing raw csv:	/media/blazaid/Saca/Phd/data/raw_csv/miguel-train-capture-can.csv ...
Dumping /capture/gps/position
Writing raw csv:	/media/blazaid/Saca/Phd/data/raw_csv/miguel-train-capture-gps-position.csv ...
Dumping /capture/gps/speed
Writing raw csv:	/media/blazaid/Saca/Phd/data/raw_csv/miguel-train-capture-gps-speed.csv ...
Dumping /capture/lidar
Writing raw csv:	/media/blazaid/Saca/Phd/data/raw_csv/miguel-train-capture-lidar.csv ...
Dumping /capture/kinect/image
Writing raw csv:	/media/blazaid/Saca/Phd/data/raw_csv/miguel-train-capture-kinect-image.csv ...
Loading bag:	/media/blazaid/Saca/Phd/data/bags/miguel-test.bag
Dumping /capture/can
Writing raw csv:	/media/blazaid/Saca/Phd/data/raw_csv/miguel-test-capture-can.csv ...
Dumping /capture/gps/position
Writing raw csv:	/media/blazaid/Saca/Phd/data/raw_csv/miguel-test-capture-gps-position.csv ...